In [1]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\home laptop\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
from numpy.random import seed
seed(1)

In [4]:
# import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [5]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [6]:
df = pd.read_csv("exoplanet_data.csv")
# df = df.drop (columns = ["koi_period_err1","koi_period_err2","koi_time0bk_err1","koi_time0bk_err2", "koi_impact_err1","koi_impact_err2", "koi_duration_err1","koi_duration_err2","koi_depth_err1","koi_depth_err2", "koi_prad_err1","koi_prad_err2", "koi_steff_err1","koi_steff_err2", "koi_slogg_err1","koi_slogg_err2","koi_srad_err1","koi_srad_err2"])
# # Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [7]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition','koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr']]
selected_features = selected_features[selected_features['koi_disposition']!='CANDIDATE']
selected_features['koi_disposition'] = selected_features['koi_disposition'].map({'CONFIRMED': 1, 'CANDIDATE': 2, 'FALSE POSITIVE':3})
selected_features

,koi_disposition,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr
0,1,0.586,4.50700,874.8,2.83,443,9.11,25.8
1,3,0.969,1.78220,10829.0,14.60,638,39.30,76.3
2,3,1.276,2.40641,8079.2,33.46,1395,891.96,505.6
3,1,0.701,1.65450,603.3,2.75,1406,926.16,40.9
4,1,0.762,3.14020,686.0,2.77,1160,427.65,40.2
...,...,...,...,...,...,...,...,...
6983,3,2.441,77.98300,1583.8,534.47,1006,242.54,171.5
6986,3,0.765,4.80600,87.7,1.11,929,176.40,8.4
6987,3,1.252,3.22210,1579.2,29.35,2088,4500.53,453.3
6989,3,0.147,0.86500,103.6,1.07,2218,5713.41,12.3


In [8]:
# Read the training data into a dataframe
X = selected_features[['koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr']]
y = selected_features['koi_disposition']

In [9]:
#columns
feature_names = selected_features.columns

# Create a Train Test Split

In [10]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [11]:
# Read the training data into a dataframe
X_train_df = pd.DataFrame(X_train)
X_train_df.head()

,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr
4608,0.5181,1.9230,195.7,1.37,386,5.23,5.2
2644,0.9320,2.2491,3860.7,11.26,926,173.50,218.5
1322,1.2720,6.7890,3522.5,32.77,954,195.79,141.4
460,0.8660,3.0822,421.6,2.41,777,86.28,24.6
2579,0.1930,8.6567,315.6,2.71,802,97.94,67.9


In [12]:
# Convert the dataframe to a numpy array for Keras
X_train = X_train_df.values
X_train.shape

(3978, 7)

In [13]:
# Read in the training labels as a dataframe
y_train_df = pd.DataFrame(y_train)
y_train.head()

4608    3
2644    1
1322    3
460     1
2579    1
Name: koi_disposition, dtype: int64

In [ ]:
y_train = y_train_df.values
X_train.shape
# One-hot encode the integer labels
# 1 CONFIRMED
# 2? CANDIDATE
# 3 FALSE POSITIVE

In [15]:
y_train = to_categorical(y_train_df)
y_train[:10]
y_train.shape

(3978, 4)

In [16]:
# Read the testing labels
y_test_df = pd.DataFrame(y_test)
# One-hot encode the integer labels
y_test = to_categorical(y_test_df)
y_test.shape

(1326, 4)

# Build a Deep Neural Network

In [17]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=150, activation='relu', input_dim=7))
model.add(Dense(units=150, activation='relu'))
model.add(Dense(units=150, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

In [ ]:
# Create an empty sequential model
# model = Sequential()

In [ ]:
# Add the first layer where the input dimensions are the 3978 columns of the training data
# model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))

In [ ]:
# Add a second hidden layer
# model.add(Dense(100, activation='relu'))

In [18]:
# The output layer has 2 columns that are one-hot encoded
y_train.shape

(3978, 4)

In [ ]:
# # Add output layer
# model.add(Dense(y_train.shape[1], activation="softmax"))

In [19]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [21]:
model.summary

<bound method Network.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002650B1F3C48>>

In [20]:
# Use the training data to fit (train) the model
model.fit(
    X_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
125/125 - 0s - loss: 43.6424 - accuracy: 0.6662
Epoch 2/100
125/125 - 0s - loss: 68.0866 - accuracy: 0.6807
Epoch 3/100
125/125 - 0s - loss: 64.5914 - accuracy: 0.6677
Epoch 4/100
125/125 - 0s - loss: 6.8264 - accuracy: 0.6905
Epoch 5/100
125/125 - 0s - loss: 5.8524 - accuracy: 0.7071
Epoch 6/100
125/125 - 0s - loss: 9.0832 - accuracy: 0.6863
Epoch 7/100
125/125 - 0s - loss: 3.3110 - accuracy: 0.7006
Epoch 8/100
125/125 - 0s - loss: 18.9086 - accuracy: 0.6948
Epoch 9/100
125/125 - 1s - loss: 2.3387 - accuracy: 0.7049
Epoch 10/100
125/125 - 0s - loss: 2.7520 - accuracy: 0.7260
Epoch 11/100
125/125 - 0s - loss: 5.1489 - accuracy: 0.7217
Epoch 12/100
125/125 - 0s - loss: 2.2042 - accuracy: 0.7386
Epoch 13/100
125/125 - 0s - loss: 3.4643 - accuracy: 0.7031
Epoch 14/100
125/125 - 0s - loss: 1.1732 - accuracy: 0.7318
Epoch 15/100
125/125 - 0s - loss: 0.7122 - accuracy: 0.7815
Epoch 16/100
125/125 - 0s - loss: 2.3049 - accuracy: 0.7486
Epoch 17/100
125/125 - 0s - loss: 3.2584 - ac

# Save the Model

In [22]:
model.save("DL.h5")

# Evaluate the Model

In [25]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("DL.h5")

In [26]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

42/42 - 0s - loss: 0.3485 - accuracy: 0.8733
Loss: 0.34851351380348206, Accuracy: 0.8733031749725342


In [30]:
X_train[0]

array([  0.5181,   1.923 , 195.7   ,   1.37  , 386.    ,   5.23  ,
         5.2   ])

In [33]:
X_test.iloc[0]

koi_impact          0.9640
koi_duration        3.0898
koi_depth         136.4000
koi_prad            1.3000
koi_teq          1384.0000
koi_insol         866.2100
koi_model_snr      18.9000
Name: 1290, dtype: float64

In [34]:
# Grab just one data point to test with
test = np.expand_dims(X_test.iloc[0], axis=0)
test.shape

(1, 7)

In [37]:
# Make a prediction. The result should be 5 - STANDING
print(f"Predicted class: {model.predict_classes(test)}")

Predicted class: [3]
